# Create Harmonization Benchmark To GDC Data Dictionary

This notebook utilizes data from the following paper:

* Yurong Liu, Eduardo H. M. Pena, Aécio Santos, Eden Wu, and Juliana Freire. 2025. Magneto: Combining Small and Large Language Models for Schema Matching. Proc. VLDB Endow. 18, 8 (April 2025), 2681–2694. https://doi.org/10.14778/3742728.3742757

## Setup

In [ ]:
# If you are actively working on related *.py files and would like changes to reload automatically into this notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json

## Get benchmark data

Set input and output directories

In [ ]:
input_dir = os.path.abspath(
    "../datasets/harmonization_benchmark_real_GDC/inputs"
)

output_dir = os.path.abspath(
    "../datasets/harmonization_benchmark_real_GDC"
)

Get GDC Data Dictionary as a target model

In [ ]:
url = "https://api.gdc.cancer.gov/v0/submission/_dictionary/_all"
target_model_path = os.path.abspath(input_dir) + "/target_model_GDC.json"
os.makedirs(os.path.dirname(target_model_path), exist_ok=True)
!wget -q -O "{target_model_path}" "{url}"

Get 10 source CSVs from the paper as a source tables

In [ ]:
source_tables = [
    ("1MyQOryVm3S0iBz3-uqAC_bPqZjMtS6IA", "Cao.csv"), # pragma: allowlist secret
    ("1N3rbTHtnVDe19kMNei0opy_g-8Hr_Hl5", "Clark.csv"), # pragma: allowlist secret
    ("1Ml-lY2LnAwpFpgHGeE7R2qqWRxBVLso9", "Dou.csv"), # pragma: allowlist secret
    ("1Nac7mZR_reZPdK5zghI5Y3pEKq8VPTRQ", "Gilette.csv"), # pragma: allowlist secret
    ("1NIFT5dHcguZ1vzbQ_qz1tIhNDx-QENSe", "Huang.csv"), # pragma: allowlist secret
    ("1MjNgXn-peUUaSadqIcWlqszECgxw7-ux", "Krug.csv"), # pragma: allowlist secret
    ("1ND-qu_62kGtz98O23AMFId4SHQX5GPzJ", "McDermott.csv"), # pragma: allowlist secret
    ("1NE13PtlXR6w2wRXyZrY6ar2lUXeLUw1-", "Satpathy.csv"), # pragma: allowlist secret
    ("1MxEwZbz-31bQqM8ECIKnrClSQNTNwwpY", "Vasaikar.csv"), # pragma: allowlist secret
    ("1NgEsOT7jPdCll0Q3iQ_tuAMqe8L2XFBE", "Wang.csv") # pragma: allowlist secret
]

source_tables_path = os.path.abspath(input_dir) + "/source_tables"
os.makedirs(source_tables_path, exist_ok=True)


for id, name in source_tables:
    url = f"https://drive.google.com/uc?export=download&id={id}"
    !wget -q --no-check-certificate "{url}" -O "{source_tables_path}/{name}"

Get 10 ground truth mappings CSVs from the paper as source mappings

In [ ]:
source_mappings = [
    ("1c64T1cq09T6WmOIIMGRO6yglIRBDDaYP", "Cao.csv"), # pragma: allowlist secret
    ("10pzRiZWuhE_jfNAm7D8XzKzM7ebJgbyj", "Clark.csv"), # pragma: allowlist secret
    ("1vqL5HhFT6SxptQu7FyLidJnn2VKb4UMg", "Dou.csv"), # pragma: allowlist secret
    ("1S0Fe2YlcqNhO1aFMwnePLjKVm1LPVDL8", "Gilette.csv"), # pragma: allowlist secret
    ("1Jy3FIE8jcrNiNlyXsoQIo86nfGVSeAsL", "Huang.csv"), # pragma: allowlist secret
    ("1VS27jhKjNjxPnxn4SJt3OcvbMItYSfG2", "Krug.csv"), # pragma: allowlist secret
    ("107WFZ_-kCY-Yh9MGn1Fx1N93b23be27D", "McDermott.csv"), # pragma: allowlist secret
    ("1JY5fo4Tg3b_bgp-6JHPqweCiunjpWqPe", "Satpathy.csv"), # pragma: allowlist secret
    ("1qZ_kOz9-iC8IzMSvdRHhZIc-mjrU-aSk", "Vasaikar.csv"), # pragma: allowlist secret
    ("1N8h2qwWBy8IO7QMx9ahkUE6vuhDdT6El", "Wang.csv") # pragma: allowlist secret
]

source_mappings_path = os.path.abspath(input_dir) + "/source_mappings"
os.makedirs(source_mappings_path, exist_ok=True)


for id, name in source_mappings:
    url = f"https://drive.google.com/uc?export=download&id={id}"
    !wget -q --no-check-certificate "{url}" -O "{source_mappings_path}/{name}"

## Format benchmark data

Some columns contain muliple numeric values seperated by semicolon (ex "1;2;3"), this function will extract numeric values from these strings

In [ ]:
def extract_numeric_from_mixed(col_values):
    flat = []
    for val in col_values.dropna():
        elements = str(val).split(";")
        for x in elements:
            try:
                num = float(x)
                flat.append(num)
            except ValueError:
                continue
    return pd.Series(flat, dtype=float)

This function creates JSONS from CSV

In [ ]:
def csv_to_schema(df, bins=5, thresh_numeric=0.5, file_name=None):
    schema = {"type": "object", "properties": []}
    if file_name:
        basename = os.path.splitext(file_name)[0]
        schema["name"] = basename
    for col in df.columns:
        col_info = {}
        col_values = df[col]
        numeric = pd.to_numeric(col_values, errors='coerce')
        col_info['name'] = col
        if numeric.notnull().sum() >= thresh_numeric * col_values.notnull().sum():
            numeric_values = extract_numeric_from_mixed(col_values)
            numeric_values = numeric_values.dropna().astype(float)
            if len(numeric_values) > 1:
                counts, bin_edges = np.histogram(numeric_values, bins=bins)
                bin_mids = ((bin_edges[:-1] + bin_edges[1:]) / 2).tolist()
            else:
                bin_mids = numeric_values.tolist()
                counts = [1] * len(numeric_values)
            col_info['type'] = 'number'
            col_info['histogram'] = {
                "bins": bin_mids,
                "counts": counts if isinstance(counts, list) else counts.tolist()
            }
        else:
            value_list = []
            for val in col_values.dropna():
                if ';' in str(val):
                    value_list.extend(str(val).split(';'))
                elif '|' in str(val):
                    value_list.extend(str(val).split('|'))
                else:
                    value_list.append(val)
            counts = pd.Series(value_list).value_counts()
            bins_ = counts.index.tolist()
            values = counts.values.tolist()
            col_info['type'] = 'string'
            col_info['histogram'] = {
                "bins": bins_,
                "counts": values
            }
        schema["properties"].append(col_info)
    return schema

Convert source CSVs to JSONs models and save source and target models in {source}_{target} folder

In [ ]:
target_name = "GDC"

for fname in os.listdir(source_tables_path):
    source_name = os.path.splitext(fname)[0]
    source_target_path = f"{os.path.abspath(output_dir)}/{source_name}_{target_name}"
    os.makedirs(source_target_path, exist_ok=True)
    if fname.endswith(".csv"):
        csv_path = os.path.join(source_tables_path, fname)
        try:
            df = pd.read_csv(csv_path)
            schema = csv_to_schema(df, bins=5, file_name=fname)
            json_path = os.path.join(source_target_path, f"source_model.json")
            with open(json_path, "w") as f:
                json.dump(schema, f, indent=2)
            print(f"Source files processed and saved: {json_path}")
            target_path = shutil.copy(target_model_path, source_target_path + "/target_model.json")
            print(f"Target model copied to: {target_path}")
        except Exception as e:
            print(f"Error processing {fname}: {e}")

Format mappings and save them in {source}_{target} folder

In [ ]:
import os
import pandas as pd
import json

# Load previously downloaded GDC dictionary
with open(target_model_path, "r") as f:
    gdc_dict = json.load(f)

attribute_to_node = {}

# Iterate all top-level keys (skip ones starting with "_", e.g. "_definitions")
for node_name, node in gdc_dict.items():
    if node_name.startswith("_"):
        continue              # these are not real data nodes
    if "properties" in node:
        for prop in node["properties"].keys():
            attribute_to_node[prop] = node_name

# For debugging
print("Sample mapping:", dict(list(attribute_to_node.items())[:10]))

for fname in os.listdir(source_mappings_path):
    if fname.lower().endswith(".csv"):
        source_name = os.path.splitext(fname)[0]
        source_target_path = f"{os.path.abspath(output_dir)}/{source_name}_{target_name}"
        os.makedirs(source_target_path, exist_ok=True)
        df = pd.read_csv(os.path.join(source_mappings_path, fname))
        src_col = [c for c in df.columns if "original" in c.lower()][0]
        trg_col = [c for c in df.columns if "gdc" in c.lower()][0]
        rows = []
        for _, row in df.iterrows():
            src = f"{source_name}.{str(row[src_col]).strip()}"
            trg_var = str(row[trg_col]).strip()
            node_prefix = attribute_to_node.get(trg_var, "")
            trg = f"{node_prefix}.{trg_var}" if node_prefix else trg_var
            rows.append({"source_node_prop_type_desc": src, "target_node_prop_type_desc": trg})
        out_df = pd.DataFrame(rows)
        out_csv = os.path.join(source_target_path, f"expected_mappings.csv")
        out_df.to_csv(out_csv, index=False)
        print(f"Mappings processed and saved: {out_csv}")

Format of output:

- output_dir
    - source_target_folder_0
        - `source_model.json`
        - `expected_mappings.tsv`
        - `target_model.json`
    - ...